<a href="https://colab.research.google.com/github/cedamusk/daily_/blob/main/Mineral_alteration_per_depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install plotly pandas scipy

In [85]:
import os
from google.colab import drive

In [86]:
def mount_google_drive():
  drive.mount('/content/drive')

In [87]:
def check_directory_contents(path):
  for item in os.listdir(path):
    print(item)

In [88]:
mount_google_drive()
dataset_path='/content/drive/MyDrive/Alteration table'
check_directory_contents(dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ALTERATION TABLE.csv


In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from scipy.interpolate import griddata

In [90]:
df=pd.read_csv('/content/drive/MyDrive/Alteration table/ALTERATION TABLE.csv')

In [91]:
# Color map for alterations
color_map = {
    'Sm.': '#FF0000',          # Red
    'Chl: ill': '#00FF00',      # Green
    'ill': '#800080',           # Purple
    'Chl:sm': '#FFFF00',        # Yellow
    'No Alteration': '#808080'  # Gray
}

def create_connectivity_visualization(df):

    df['LONGITUDE']=pd.to_numeric(df['LONGITUDE'], errors='coerce')
    df['LATITUDE']=pd.to_numeric(df['LATITUDE'], errors='coerce')
    df['DEPTH']=pd.to_numeric(df['DEPTH'], errors='coerce')
    # Prepare data for interpolation
    longitude = df['LONGITUDE']
    latitude = df['LATITUDE']
    depth = df['DEPTH']
    alterations = df['MINERAL ALTERATION']

    # Create a dense grid for interpolation
    grid_longitude = np.linspace(longitude.min(), longitude.max(), 100)
    grid_latitude = np.linspace(latitude.min(), latitude.max(), 100)
    grid_longitude, grid_latitude = np.meshgrid(grid_longitude, grid_latitude)

    # Interpolation for each unique alteration type
    unique_alterations = df['MINERAL ALTERATION'].unique()
    unique_alterations = [alt for alt in unique_alterations if alt not in ['No clear peaks', 'No Alteration']]

    fig = go.Figure()

    for alt_type in unique_alterations:
        # Filter data for specific alteration type
        alt_df = df[df['MINERAL ALTERATION'] == alt_type]
        if len(alt_df)<4:
          print(f"Skipping interpolation for {alt_type} due to insufficient data points.")
          continue
        alt_df['LONGITUDE_jittered']=alt_df['LONGITUDE']+np.random.normal(0,0.0001, len(alt_df))
        alt_df["LATITUDE_jittered"]=alt_df['LATITUDE']+np.random.normal(0, 0.0001, len(alt_df))
        # Interpolate depth for this alteration type
        grid_depth = griddata(
            (alt_df['LONGITUDE_jittered'], alt_df['LATITUDE_jittered']),
            alt_df['DEPTH'],
            (grid_longitude, grid_latitude),
            method='nearest'
        )

        # Create surface for this alteration type
        surface = go.Surface(
            x=grid_longitude,
            y=grid_latitude,
            z=grid_depth,
            colorscale=[[0, color_map.get(alt_type, '#000000')]],
            opacity=0.6,
            name=alt_type,
            showscale=False
        )
        fig.add_trace(surface)

    # Add original data points
    for alt_type in unique_alterations:
        alt_df = df[df['MINERAL ALTERATION'] == alt_type]
        scatter = go.Scatter3d(
            x=alt_df['LONGITUDE'],
            y=alt_df['LATITUDE'],
            z=alt_df['DEPTH'],
            mode='markers',
            name=f'{alt_type} Points',
            marker=dict(
                size=5,
                color=color_map.get(alt_type, '#000000'),
                opacity=1
            )
        )
        fig.add_trace(scatter)

    # Customize layout
    fig.update_layout(
        title='Mineral Alteration Connectivity Surface',
        scene=dict(
            xaxis_title='Longitude',
            yaxis_title='Latitude',
            zaxis_title='Depth',
            zaxis=dict(autorange='reversed')
        ),
        height=800,
        width=1200
    )

    return fig

# Create and show the visualization
fig = create_connectivity_visualization(df)
fig.show()

# Save the interactive HTML
fig.write_html('/content/mineral_alteration_connectivity.html')

Skipping interpolation for Chl: ill  due to insufficient data points.
Skipping interpolation for ill  due to insufficient data points.


<ipython-input-91-2f5ca98bda94>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-91-2f5ca98bda94>:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-91-2f5ca98bda94>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-91-2f5c

In [92]:
def process_mineral_alteration(alteration):
  if pd.isna(alteration) or alteration=='No Clear peaks':
    return 'No Alteration'
  return alteration.strip()

In [93]:
df['Processed_Alteration']=df['MINERAL ALTERATION'].apply(process_mineral_alteration)

In [94]:
unique_alterations=df["Processed_Alteration"].unique()
color_map={
    'Sm.': '#FF0000',
    'Chl:ill':'#00FF00',
    'ill':'#800080',
    'Chl:sm':'#FFFF00',
    'No Alteration':'#808080'
}

In [95]:
def prepare_interpolation_data(df, alteration_type):
  alt_df=df[df['MINERAL ALTERATION']==alteration_type]

  longitude=alt_df['LONGITUDE']
  latitude=alt_df['LATITUDE']
  depth=alt_df['DEPTH']

  if len(longitude.unique())<2 or len(latitude.unique())<2:
    grid_longitude=np.array([])
    grid_latitude=np.array([])
    grid_depth=np.array([])
    return grid_longitude, grid_latitude, grid_depth

  alt_df=alt_df.drop_duplicates(subset=['LONGITUDE', 'LATITUDE'], keep='first')
  longitude=alt_df['LONGITUDE']
  latitude=alt_df['LATITUDE']
  depth=alt_df['DEPTH']

  grid_longitude=np.linspace(longitude.min(), longitude.max(), 50)
  grid_latitude=np.linspace(latitude.min(), latitude.max(), 50)
  grid_longitude, grid_latitude=np.meshgrid(grid_longitude, grid_latitude)

  grid_depth=griddata((longitude, latitude), depth,
                      (grid_longitude, grid_latitude),
                      method='nearest')

  return grid_longitude, grid_latitude, grid_depth

In [96]:
def create_interpolation_visualization(df):
  color_map={
      'Sm.':'#FF0000',
      'Chl: ill':'#00FF00',
      'ill':'#800080',
      'Chl:sm':'#FFFF00',
      'No Alteration':'#808080'
  }
  alteration_types=df['MINERAL ALTERATION'].unique()
  alteration_types=[alt for alt in alteration_types
                    if alt not in ['No clear peaks', 'No alteration']]

  fig=go.Figure()

  for alt_type in alteration_types:
    grid_longitude, grid_latitude, grid_depth=prepare_interpolation_data(df, alt_type)

    surface=go.Surface(
        x=grid_longitude,
        y=grid_latitude,
        z=grid_depth,
        colorscale=[[0, color_map.get(alt_type, '#000000')],
                    [1, color_map.get(alt_type, '#000000')]],
        opacity=0.6,
        name=alt_type,
        showscale=False
    )
    fig.add_trace(surface)

    for alt_type in alteration_types:
      grid_longitude, grid_latitude, grid_depth=prepare_interpolation_data(df, alt_type)

      surface=go.Surface(
          x=grid_longitude,
          y=grid_latitude,
          z=grid_depth,
          colorscale=[[0, color_map.get(alt_type, '#000000')],
                      [1, color_map.get(alt_type, '#000000')]],
          opacity=0.6,
          name=alt_type,
          showscale=False
      )
      fig.add_trace(surface)

  for alt_type in alteration_types:
    alt_df=df[df['MINERAL ALTERATION']==alt_type]
    scatter=go.Scatter3d(
        x=alt_df['LONGITUDE'],
        y=alt_df['LATITUDE'],
        z=alt_df['DEPTH'],
        mode='markers',
        name=f'{alt_type} Points',
        marker=dict(
            size=5,
            color=color_map.get(alt_type, '#000000'),
            opacity=1
        )
    )
    fig.add_trace(scatter)

  fig.update_layout(
      title='Mineral Alteration Interpolation',
      scene=dict(
          xaxis_title='Longitude',
          yaxis_title='Latitude',
          zaxis_title='Depth',
          zaxis=dict(autorange='reversed')
      ),
      height=800,
      width=1200
  )
  return fig

fig=create_interpolation_visualization(df)
fig.show()

In [97]:
fig=go.Figure(data=[
    go.Scatter3d(
        x=df[df['Processed_Alteration']==alt]['LONGITUDE'],
        y=df[df['Processed_Alteration']==alt]['LATITUDE'],
        z=df[df['Processed_Alteration']==alt]['DEPTH'],
        mode='markers',
        name=alt,
        marker=dict(
            size=5,
            color=color_map.get(alt, '#000000'),
            opacity=0.8
        ),
        hovertemplate=
        '<b>Longitude</b>:%{x}<br>'+
        '<b>Latitude</b>:%{y}<br>'+
        '<b>Depth</b>:%{z}<br>'+
        '<b>Alteration</b>:'+alt+'<extra></extra>'

    )for alt in unique_alterations
])

In [98]:
fig.update_layout(
    title='Mineral Alteration 3D Visualization',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Depth',
        zaxis=dict(autorange='reversed')
    ),
    height=800,
    width=1200
)

In [99]:
fig.show()